# The undirected version of the Chinese postman problem
Alicia Fernández Martínez 905092



In graph theory, the Chinese Postman Problem (CPP) addresses a question:
"Given a weighted graph, what is the shortest closed path that traverses each edge of the graph at least once?"

This problem, named after a Chinese mathematician, Mei-Ko Kwan, models real-world scenarios such as mail delivery routes, garbage collection paths, or street-sweeping plans. However, when the graph is undirected, the solution involves specific adaptations, which makes it a variant called the Undirected Chinese Postman Problem (UCPP).


In [9]:
from itertools import combinations  # Import itertools to generate node pairs
import heapq  # Import heapq for Dijkstra's algorithm

# Function to find an Eulerian tour in a graph
def find_eulerian_tour(graph):
    # Calculate the degree of each node (number of neighbors)
    degree = {node: len(neighbors) for node, neighbors in graph.items()}

    # Find nodes with an odd degree
    odd_degree_nodes = [node for node, deg in degree.items() if deg % 2 == 1]

    # If there are odd-degree nodes, the graph is not Eulerian
    if odd_degree_nodes:
        raise ValueError("Graph is not Eulerian")  # Raise an error if the graph is not Eulerian

    # Initialize the Eulerian tour and the stack
    tour = []
    stack = [next(iter(graph))]  # Stack for performing the tour (starts with any arbitrary node)

    while stack:
        node = stack[-1]  # Current node (last in the stack)

        # If the current node has unvisited neighbors
        if graph[node]:
            next_node, _ = graph[node].pop()  # Take a neighbor and remove it from connections
            graph[next_node].remove((node, _))  # Remove the reverse edge
            stack.append(next_node)  # Push the neighbor onto the stack
        else:
            tour.append(stack.pop())  # If no neighbors left, add the node to the tour

    return tour  # Return the Eulerian tour


# Function to solve the Chinese Postman Problem
def chinese_postman(graph):
    # Calculate the degree of each node
    degree = {node: len(neighbors) for node, neighbors in graph.items()}

    # Find nodes with an odd degree
    odd_degree_nodes = [node for node, deg in degree.items() if deg % 2 == 1]

    # If there are no odd-degree nodes, the graph is already Eulerian
    if not odd_degree_nodes:
        return find_eulerian_tour(graph.copy()), 0  # Find the Eulerian tour and return cost 0

    # Generate all possible pairs of odd-degree nodes
    pairs = list(combinations(odd_degree_nodes, 2))

    # Calculate the distances between each pair of odd-degree nodes using Dijkstra's algorithm
    distances = {(u, v): single_source_shortest_paths(graph, u)[v] for u, v in pairs}

    match = []  # List to store the matched pairs
    used = set()  # Set to track nodes that have already been used

    # Match nodes with the smallest distance between them
    for u, v in sorted(distances, key=distances.get):
        if u not in used and v not in used:  # If both nodes are not already matched
            match.append((u, v))  # Match the nodes
            used.update({u, v})  # Mark the nodes as used

    # Add additional edges to match odd-degree nodes
    for u, v in match:
        graph[u].append((v, distances[(u, v)]))  # Add the edge to the graph
        graph[v].append((u, distances[(u, v)]))  # Add the reverse edge (undirected)

    # Find the Eulerian tour in the modified graph
    tour = find_eulerian_tour(graph.copy())

    # Calculate the additional cost of the added edges
    additional_cost = sum(distances[(u, v)] for u, v in match)

    return tour, additional_cost


# Helper function to create a graph from edges (for the examples)
def graph_from_edges(edges, directed=False):
    graph = {}
    for u, v, weight in edges:
        if u not in graph:
            graph[u] = []
        if v not in graph:
            graph[v] = []
        graph[u].append((v, weight))  # Add the edge with its weight
        if not directed:
            graph[v].append((u, weight))  # Add the reverse edge if it's undirected
    return graph

# Dijkstra's algorithm to calculate shortest paths from a source node to all others
def single_source_shortest_paths(graph, start):
    distances = {node: float('inf') for node in graph}  # Initialize distances to infinity
    distances[start] = 0  # Distance from start node to itself is 0
    priority_queue = [(0, start)]  # Priority queue for Dijkstra's algorithm

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)  # Pop the closest node
        if current_distance > distances[current_node]:  # If current distance is greater than known distance, skip
            continue
        for neighbor, weight in graph[current_node]:  # Check neighbors of current node
            distance = current_distance + weight
            if distance < distances[neighbor]:  # If a shorter path is found
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))  # Add the neighbor with its new distance

    return distances  # Return a dictionary with the shortest distances from the start node


# Examples of usage

In [10]:
#1
graph_weighted = graph_from_edges([(0, 1, 2), (1, 2, 3), (2, 3, 1), (3, 4, 4), (4, 5, 2), (5, 0, 3)], directed=False)
postman_tour, added_cost = chinese_postman(graph_weighted)
print("Chinese Postman Tour:", postman_tour)
print("Additional cost:", added_cost)

#2
graph_weighted_2 = graph_from_edges([
    (0, 1, 4), (1, 2, 6), (2, 3, 5), (3, 4, 7), (4, 0, 3), (1, 3, 2)
], directed=False)
postman_tour_2, added_cost_2 = chinese_postman(graph_weighted_2)
print("Chinese Postman Tour:", postman_tour_2)
print("Additional cost:", added_cost_2)

#3 (large example)
graph_weighted_large = graph_from_edges([
    (0, 1, 10), (0, 2, 15), (0, 3, 20),
    (1, 2, 35), (1, 4, 25), (1, 5, 30),
    (2, 3, 40), (2, 6, 50), (3, 7, 45),
    (4, 5, 5), (4, 6, 10), (5, 7, 20),
    (6, 7, 25), (6, 8, 30), (7, 8, 35),
    (8, 9, 40), (7, 9, 20), (5, 9, 15)
], directed=False)

postman_tour_large, added_cost_large = chinese_postman(graph_weighted_large)

print("Chinese Postman Tour (Large Graph):", postman_tour_large)
print("Additional cost (Large Graph):", added_cost_large)

Chinese Postman Tour: [0, 1, 2, 3, 4, 5, 0]
Additional cost: 0
Chinese Postman Tour: [0, 1, 2, 3, 1, 3, 4, 0]
Additional cost: 2
Chinese Postman Tour (Large Graph): [0, 1, 4, 5, 1, 2, 3, 7, 5, 9, 7, 6, 4, 9, 8, 7, 8, 6, 2, 0, 3, 0]
Additional cost (Large Graph): 75
